In [ ]:
# Scientific Computing
import numpy as np
import sympy as sp

# PyCalphad (Thermodynamics Calculations & Plotting)
from pycalphad import Database, calculate
from pycalphad.plot.utils import phase_legend

# Custom
from symbolic_hulls import *

In [ ]:
def fit_binary_HSX_surface(x_data, s_data, h_data):
    # Construct the design matrix
    A = np.column_stack([
        np.ones_like(x_data),    
        x_data,                  
        s_data,                  
        x_data**2,               
        s_data**2,               
    ])
    
    # Solve the least squares problem to get the coefficients
    coeffs, residuals, rank, _ = np.linalg.lstsq(A, h_data, rcond=None)
    
    # Create sympy symbols for x and y
    x, s = sp.symbols('x s')
    
    # Define the list of polynomial terms in the same order as in A:
    terms = [
        1,          
        x,          
        s,          
        x**2,       
        s**2,       
    ]
    
    # Build the polynomial expression by summing coeff * term for each term.
    expr = sum(sp.Float(coeff) * term for coeff, term in zip(coeffs, terms))
    
    return sp.simplify(expr), residuals

In [ ]:
def fit_ternary_HSX_surface(x1_data, x2_data, s_data, h_data):
    # Construct the design matrix
    A = np.column_stack([
        np.ones_like(x1_data),    
        x1_data,
        x2_data,
        s_data,
        x1_data**2,
        x2_data**2,
        s_data**2,             
    ])
    
    # Solve the least squares problem to get the coefficients
    coeffs, residuals, rank, _ = np.linalg.lstsq(A, h_data, rcond=None)
    
    # Create sympy symbols for x and y
    x1, x2, s = sp.symbols('x1 x2 s')
    
    # Define the list of polynomial terms in the same order as in A:
    terms = [
        1,
        x1,
        x2,
        s,
        x1**2,
        x2**2,
        s**2,
    ]
    
    # Build the polynomial expression by summing coeff * term for each term.
    expr = sum(sp.Float(coeff) * term for coeff, term in zip(coeffs, terms))
    
    return sp.simplify(expr), residuals

In [ ]:
# Calculate all the enthalpy as a funciton of the entropy and composition
def format_enthalpy_binary(entropy_result, enthalpy_result):
    X = entropy_result.X.sel(component='FE').values[0, 0, :, :].flatten()
    S = entropy_result.SM.values[0, 0, :, :].flatten()
    H = enthalpy_result.HM.values[0, 0, :, :].flatten()

    sort_idx = np.argsort(X)
    X_sorted = X[sort_idx]
    H_sorted = H[sort_idx]
    S_sorted = S[sort_idx]

    sort_idx = np.argsort(S)
    X_sorted = X_sorted[sort_idx]
    S_sorted = S_sorted[sort_idx]
    H_sorted = H_sorted[sort_idx]

    return X_sorted, S_sorted, H_sorted

In [ ]:
# Calculate all the enthalpy as a funciton of the entropy and composition
def format_enthalpy_ternary(entropy_result, enthalpy_result):
    X1 = entropy_result.X.sel(component='FE').values[0, 0, :, :].flatten()
    X2 = entropy_result.X.sel(component='AL').values[0, 0, :, :].flatten()
    S = entropy_result.SM.values[0, 0, :, :].flatten()
    H = enthalpy_result.HM.values[0, 0, :, :].flatten()

    sort_idx = np.argsort(X1)
    X1_sorted = X1[sort_idx]
    X2_sorted = X2[sort_idx]
    H_sorted = H[sort_idx]
    S_sorted = S[sort_idx]

    sort_idx = np.argsort(X2)
    X1_sorted = X1_sorted[sort_idx]
    X2_sorted = X2_sorted[sort_idx]
    H_sorted = H_sorted[sort_idx]
    S_sorted = S_sorted[sort_idx]

    sort_idx = np.argsort(S)
    X1_sorted = X1_sorted[sort_idx]
    X2_sorted = X2_sorted[sort_idx]
    H_sorted = H_sorted[sort_idx]
    S_sorted = S_sorted[sort_idx]

    return X1_sorted, X2_sorted, S_sorted, H_sorted

#### Binary System

In [ ]:
# Load database and choose the phases that will be plotted
db = Database(r'../TDDatabaseFiles_temp/alfe.tdb')

phases = list(db.phases.keys())
constituents = list(db.elements)
legend_handles, color_dict = phase_legend(phases)

# These are the phases that don't have a good fit
for phase in phases:
    if phase in ['AL2FE', 'AL13FE4', 'AL5FE2']:
        phases.remove(phase)

print("Phases in Database:", phases)
print("Elements in Database:", constituents)

In [ ]:
# Here we need to compute the HSX data for the binary system
temp_points_count = 60
enthalpy_phase_dict = dict()
for phase in phases:
    print(phase)
    entropy_result = calculate(db, constituents, phase, P=101325, T=np.linspace(300, 2000, temp_points_count), output = "SM")
    enthalpy_result = calculate(db, constituents, phase, P=101325, T=np.linspace(300, 2000, temp_points_count), output = "HM")

    X, S, H = format_enthalpy_binary(entropy_result, enthalpy_result)
    enthalpy_phase_dict[phase] = (X, S, H)

In [ ]:
# Keep only the equilibrium enthalpy points for each phase
# We get these points by taking the lower convex hull of all the enthalpies
eq_enthalpy_phase_dict = dict()
for phase in phases:
    print(phase)
    X, S, H = enthalpy_phase_dict[phase]

    # Put the points into the lower hull function
    points = np.column_stack((X, S, H))
    simplices = lower_convex_hull(points)

    # Keep only the points that are in the lower hull
    points = points[np.unique(simplices.ravel())]
    eq_enthalpy_phase_dict[phase] = (points[:, 0], points[:, 1], points[:, 2])

In [ ]:
# Fit polynomials, it doesnt matter if they are a bad fit
phase_poly_dict = dict()
for phase_name in phases:
    X, S, H = eq_enthalpy_phase_dict[phase_name]

    # Compute the fitted polynomial
    energy_polynomial, res = fit_binary_HSX_surface(X, S, H)
    phase_poly_dict[phase_name] = energy_polynomial
    print(phase_name)
    display(energy_polynomial)
    print("residual:", res)

In [ ]:
save_sympy_dict_to_json_repr(phase_poly_dict, 'binary_polynomials.json')

#### Ternary System

In [ ]:
# Load database (ensure path is correct)
db = Database(r'../TDDatabaseFiles_temp/Al-Fe-O_Lindwall_etal.TDB')

# Extract available phases and elements
phases = list(db.phases.keys())  
constituents = list(db.elements)

print("Phases in Database:", phases)
print("Elements in Database:", constituents)

In [ ]:
# Here we need to compute the HSX data for the ternary system
temp_points_count = 60
enthalpy_phase_dict = dict()
for phase in phases:
    print(phase)
    entropy_result = calculate(db, constituents, phase, P=101325, T=np.linspace(300, 2000, temp_points_count), output = "SM")
    enthalpy_result = calculate(db, constituents, phase, P=101325, T=np.linspace(300, 2000, temp_points_count), output = "HM")

    X1, X2, S, H = format_enthalpy_ternary(entropy_result, enthalpy_result)
    enthalpy_phase_dict[phase] = (X1, X2, S, H)

The equilibrium code with the lower hull takes too long for the ternary system. For the moment it doesnt matter if the polynomials are a bad fit. I can try calculating the thermochemical data with equilibrium instead of calculate in the future.

In [ ]:
# # Keep only the equilibrium enthalpy points for each phase
# # We get these points by taking the lower convex hull of all the enthalpies
# eq_enthalpy_phase_dict = dict()
# for phase in phases:
#     print(phase)
#     X1, X2, S, H = enthalpy_phase_dict[phase]

#     # Put the points into the lower hull function
#     points = np.column_stack((X1, X2, S, H))
#     simplices = lower_convex_hull(points)

#     # Keep only the points that are in the lower hull
#     points = points[np.unique(simplices.ravel())]
#     eq_enthalpy_phase_dict[phase] = (points[:, 0], points[:, 1], points[:, 2], points[:, 3])

eq_enthalpy_phase_dict = enthalpy_phase_dict

In [ ]:
# Fit polynomials, it doesnt matter if they are a bad fit
phase_poly_dict = dict()
for phase_name in phases:
    X1, X2, S, H = eq_enthalpy_phase_dict[phase_name]

    # Compute the fitted polynomial
    energy_polynomial, res = fit_ternary_HSX_surface(X1, X2, S, H)
    phase_poly_dict[phase_name] = energy_polynomial
    print(phase_name)
    display(energy_polynomial)
    print("residual:", res)

In [ ]:
save_sympy_dict_to_json_repr(phase_poly_dict, 'ternary_polynomials.json')